In [ ]:
env = 'colab' # 'local' or 'colab'

In [ ]:
if env == 'colab':  
    !pip install ipynb
    !pip install huggingface-hub
    !pip install datasets
    !pip install transformers
    !pip install umap-learn
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import transformers
import torch
from tqdm.notebook import tqdm
import gc

In [ ]:
from Daily_Dialog_loader import get_DailyDialog #custom function

In [ ]:
df_train = get_DailyDialog()

Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/4.48M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset daily_dialog downloaded and prepared to /root/.cache/huggingface/datasets/daily_dialog/default/1.0.0/c03444008e9508b8b76f1f6793742d37d5e5f83364f8d573c2747bff435ea55c. Subsequent calls will reuse this data.


In [ ]:
df_train.head()

,utter_hashed,utter,dialog_id,utter_in_dialog
0,-9222904206152934825,"Well , I do prefer the lower emission cars as...",[2424],[3]
1,-9222750093521621548,I've got to get dad a Father's Day gift .,[6773],[0]
2,-9222639987932711106,"Yes , thank you . I am really far behind on t...",[10395],[1]
3,-9222533402910617688,"With iced water , please .",[2124],[3]
4,-9222336008188802012,"Oh , I don ’ t think he wants to . He wants t...","[1278, 1741]","[3, 6]"


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [ ]:
special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
df_train.utter = df_train.utter.apply(lambda x: x.lower())
#tokenized = df_train.utter.apply(lambda x: tokenizer.encode(x, add_special_tokens=True))


In [ ]:
tok = tokenizer.batch_encode_plus([u for u in df_train.utter], pad_to_max_length=True, return_tensors='pt')

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     

In [ ]:
tok['input_ids'].shape

torch.Size([72462, 291])

In [ ]:
def DialoGPT_enc(tokens_pad, att_mask):
  input_ids = tokens_pad
  attention_mask = att_mask
  batch_size = 10
  output = []

  for idx in range(0, len(input_ids), batch_size):
    if idx + batch_size <= len(input_ids):
      batch = input_ids[idx:idx + batch_size].to(device)
      part_attention_mask = attention_mask[idx:idx + batch_size].to(device)
    else:
      batch = input_ids[idx:].to(device)
      part_attention_mask = attention_mask[idx:].to(device)  

    with torch.no_grad():
      last_hidden_states = model(batch, attention_mask=part_attention_mask)
      output.append(last_hidden_states[0].cpu()[:,0,:])
      gc.collect()
  return torch.cat(output,dim = 0)

In [ ]:
import pickle

In [ ]:
outer_batch = 50
l_inp = len(tok['input_ids'])
out_big = []
#batch_numb=1
for idx in tqdm(range(0, l_inp, outer_batch)):
    if idx + outer_batch <= l_inp:
      input_ids = tok['input_ids'][idx:idx + outer_batch]
      attention_mask = tok['attention_mask'][idx:idx + outer_batch]
    else:
      input_ind = tok['input_ids'][idx:]
      attention_mask = tok['attention_mask'][idx:]
    out= DialoGPT_enc(input_ids, attention_mask)
    with open('/content/drive/My Drive/NN-for-text/data/DialogGPT_large_emb.pkl', 'ab') as out_file:
      pickle.dump(out.numpy(),out_file)
    out_big.append(out)

In [ ]:
joined = torch.cat(out_big)

In [ ]:
joined.shape

torch.Size([72450, 50260])

### Clustering

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
knn = NearestNeighbors(n_neighbors=15)
knn.fit(joined)

NearestNeighbors(n_neighbors=15)

In [ ]:
df_train[df_train['utter'] == ' Does it bother you when you have to chew a lot ? '.lower()]

,utter_hashed,utter,dialog_id,utter_in_dialog
20189,-4147473204590772099,does it bother you when you have to chew a lo...,[7344],[8]


In [ ]:
_, neighbours = knn.kneighbors([joined[20189].numpy()])
neighbours = df_train.iloc[neighbours[0].tolist()]['utter']
for n in neighbours:
  print(f"|{n}|")

| does it happen often ? |
| does it come with a frame ? |
| does your present employer know you are looking for another job ? |
| does he even know how to speak english ? |
| does that mean the stronger the better ? |
| does it have enough lodging ? |
| does that include the service charge ? |
| does the hotel run its own laundry service ? |
| does it include bonuses ? |
| does 7 o'clock suit you ? |
| does this fashion really fit my figure ? |
| does it include the charge for drinks ? |
| does it bite or scratch ? |
| does the rent include utilities ? |
| does he do that with his dad ? i mean , does he complain things to his father ? |
